In [2]:
# -*- coding: utf-8 -*-
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import unittest, time, re
import csv
import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [4]:
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "localhost:9222")
chrome_driver = "/home/kimjadong2020/Documents/chromedriver"
driver = webdriver.Chrome(chrome_driver, options=chrome_options)

In [14]:
driver.implicitly_wait(30)

In [6]:
#"https://accounts.google.com/o/oauth2/v2/auth/identifier?client_id=744908910548-j830dbmnnjs6kmro0u04eno1j1mbhnnn.apps.googleusercontent.com&response_type=code&scope=profile&redirect_uri=https%3A%2F%2Frivalregions.com%2Frival%2Fgoogles&state=8d2915a09674341eb422f937714dc67b&flowName=GeneralOAuthFlow"
url= "https://rivalregions.com/#overview"
driver.get(url)

In [7]:
# training war
driver.get("https://rivalregions.com/#overview")
driver.find_element_by_xpath("//div[@action='war']").click()
driver.find_element_by_xpath("//div[@id='content']/div[4]/div[2]/div").click()
driver.find_element_by_xpath("//div[@id='send_b_wrap']/div[4]").click()
driver.find_element_by_class_name("header_logo_inner").click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id='content']/div[4]/div[2]/div"}
  (Session info: chrome=86.0.4240.75)


In [34]:
# 홈 버튼
driver.find_element_by_class_name("header_logo_inner").click()

driver = webdriver.Chrome()

In [12]:
def goStorage():
    url="https://rivalregions.com/#storage"
    driver.get(url)
    driver.implicitly_wait(30)
    driver.refresh()

In [24]:
goStorage()

In [8]:
# 총 재산 체크
money = driver.find_element_by_xpath("/html/body/div[5]/div[1]/div[1]/div[4]/span[1]/span").text
print(f"총 재산 : {money}")
print(money.replace(".","",4))

총 재산 : 658.487.849
658487849


In [14]:
def csvAppend (priceList):
    with open("price_list.csv", "a", newline='') as f:
        writer = csv.writer(f)
        writer.writerow( priceList )

In [15]:
def priceLoging (numList):
    priceList = []
    now = str( datetime.datetime.now() )
    priceList.append(now)
    
    for num in numList:
        s_xpath = "//div[@url='"+ str(num) +"']"
        driver.find_element_by_xpath( s_xpath ).click() # 자원 클릭
        time.sleep(2)
        element_price = driver.find_element_by_xpath("/html/body/div[6]/div[1]/div[1]/div[2]/div[1]/div[3]/span/span")
        price = int(element_price.text.split(" ")[0].replace(".",""))
        priceList.append(price)
        
    csvAppend( priceList )

In [58]:
itemList = [3, 4, 11, 15, 26] # 석유 광물 우라늄 다이아 라이벌륨

In [60]:
priceLoging(itemList)

In [51]:
with open("price_list.csv", "w", newline='') as f:
    writer = csv.writer(f)
    column = ["time", "oil", "ore", "uranium", "diamonds", "rivalium"]
    writer.writerow( column )

In [125]:
s_xpath = "//div[@url='26']"
driver.find_element_by_xpath( s_xpath ).click() # 자원 클릭
element_price = driver.find_element_by_xpath("/html/body/div[6]/div[1]/div[1]/div[2]/div[1]/div[3]/span/span")
price = int(element_price.text.split(' ')[0].replace(".",""))        
print(price)

169990


In [ ]:
# 군사학교
driver.find_element_by_xpath("//div[@id='quests_wrap']/div").click()
driver.find_element_by_xpath("//div[@id='content']/div[5]/div").click()
driver.find_element_by_xpath("//div[@id='header_slide_inner']/div[3]/div[2]").click()
driver.find_element_by_id("slide_close").click()

In [82]:
def refillGold():
    url = "https://rivalregions.com/#parliament/offer"
    driver.get(url)
    driver.refresh()
    time.sleep(5)
    driver.find_element_by_xpath("//div[@id='offer_dd']/div/div/div").click()
    time.sleep(2)
    driver.find_element_by_link_text("Resources exploration: state").click()
    driver.find_element_by_id("offer_do").click()

In [120]:
def goldRefiiler(): # 금 채우기 검사 및 실행
    url = "https://rivalregions.com/#listed/stateresources/3330"
    driver.get(url)
    driver.refresh()
    time.sleep(3)

    gold_reserve = driver.find_elements_by_class_name("list_level")

    tempList = []
    for element in gold_reserve:
        tempList.append(float(element.text))

    array_gold = np.array(tempList).reshape(-1, 4)

    df_gold = pd.DataFrame(
        array_gold, 
       columns=["Explored", "Maximum", "Deep_exploration", "Limit_left"]
    )

    str_expr = "Limit_left > 0 and Explored < Maximum - 50"
    df_q = df_gold.query(str_expr) 

    if df_q.empty:
        print("채우지 않음")
    else:
        print("do refill")
        refillGold()

In [115]:
def budgetCheck():
    url = "https://rivalregions.com/#state/details/3330"
    driver.get(url)
    driver.refresh()
    time.sleep(4)

    itemList = []

    budgetList = driver.find_element_by_css_selector(
        "#header_slide_inner > div.minwidth > div.slide_profile_photo > div.imp").find_elements_by_class_name("tip")

    for i in budgetList:
        item = int(i.text.split(" ")[0].replace(".",""))
        itemList.append(item)

    df = pd.DataFrame(
        np.array(itemList).reshape(1, 6), 
        columns=["cash", "gold", "oil", "ore", "uranium", "diamond"]
    ).T
    return df

In [116]:
df_previous_budget = budgetCheck()

In [117]:
goldRefiiler()

do refill


In [118]:
df_budget = budgetCheck()

In [119]:
df2_diff = df_budget - df_previous_budget

In [113]:
df_diff

,0
cash,-2164748073
gold,-2164748073
oil,-64032
ore,-64032
uranium,-2634
diamond,-44


In [121]:
df2_diff

,0
cash,138386499
gold,213539386
oil,0
ore,0
uranium,0
diamond,0


In [22]:
req = requests.get(url)

In [23]:
html = req.text

In [24]:
with open("test.html", "w", newline='', encoding="utf-8") as f:
    f.write( html )